In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

In [2]:
batch_size = 1
INPUT_SIZE = 224

image_gen_train = ImageDataGenerator(rescale=1./255, 
                                     rotation_range=15, 
                                     width_shift_range=.15, 
                                     height_shift_range=.15, 
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     zoom_range=0.2
                                    )


train_data_gen = image_gen_train.flow_from_directory(directory='./products_images/train',
                                                     class_mode='categorical', 
                                                     target_size=(INPUT_SIZE, INPUT_SIZE),
                                                     batch_size=batch_size)

Found 310 images belonging to 5 classes.


In [3]:
train_data_gen.class_indices

{'iphone12_pro_max': 0,
 'jbl_charge3': 1,
 'nintendo_switch': 2,
 'ps4_controller': 3,
 'yeezy_boost_350': 4}

In [4]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(directory='./products_images/validate', 
                                                   class_mode='categorical', 
                                                   target_size=(INPUT_SIZE, INPUT_SIZE),
                                                   batch_size=batch_size)

Found 113 images belonging to 5 classes.


In [5]:
val_data_gen.class_indices

{'iphone12_pro_max': 0,
 'jbl_charge3': 1,
 'nintendo_switch': 2,
 'ps4_controller': 3,
 'yeezy_boost_350': 4}

In [6]:
transfer_learning_layer = tf.keras.applications.MobileNetV2(include_top=False, 
                                                      weights='imagenet', 
                                                      input_shape=(INPUT_SIZE, INPUT_SIZE, 3)
                                                     )

In [7]:
transfer_learning_layer.trainable = False

In [8]:
model = Sequential([
    transfer_learning_layer,
    Dropout(0.2),
    Flatten(),
    Dense(5)
])

In [9]:
model.compile(optimizer='Adam', 
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0), 
              metrics=['accuracy']
             )

In [10]:
epochs = 4
history = model.fit(train_data_gen, 
                    epochs=epochs, 
                    validation_data=val_data_gen)

Epoch 1/4
310/310 [==============================] - 12s 28ms/step - loss: 5.9016 - accuracy: 0.6871 - val_loss: 3.1120 - val_accuracy: 0.8850
Epoch 2/4
310/310 [==============================] - 8s 24ms/step - loss: 2.9443 - accuracy: 0.8710 - val_loss: 1.6567 - val_accuracy: 0.9469
Epoch 3/4
310/310 [==============================] - 8s 25ms/step - loss: 2.5437 - accuracy: 0.9097 - val_loss: 0.9561 - val_accuracy: 0.9381
Epoch 4/4
310/310 [==============================] - 8s 25ms/step - loss: 3.1357 - accuracy: 0.9323 - val_loss: 2.0985 - val_accuracy: 0.9292


In [11]:
model.save('.\extracted_product_detection_model')

C:\Users\Gunji\Anaconda3\envs\data_science_projects\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: .\extracted_product_detection_model\assets


INFO:tensorflow:Assets written to: .\extracted_product_detection_model\assets


In [12]:
image_gen_test = ImageDataGenerator(rescale=1./255)

ps4_controller_test_data_gen = image_gen_test.flow_from_directory(directory='./products_images/test', 
                                                   class_mode='categorical', 
                                                   target_size=(INPUT_SIZE, INPUT_SIZE),
                                                   batch_size=batch_size)

Found 8 images belonging to 5 classes.


In [13]:
test_pred = model.predict(ps4_controller_test_data_gen)
test_pred

array([[-168.09169  , -106.938934 ,  -47.878033 ,  106.91432  ,
         -51.441925 ],
       [-158.89769  ,   31.668007 ,  -40.66683  ,  -80.12598  ,
         -75.5634   ],
       [-149.68968  , -167.94661  ,  -93.22367  ,   76.06532  ,
         -51.29579  ],
       [-214.37602  , -148.7279   ,   -0.6977628,   94.38679  ,
         -79.140785 ],
       [-204.69273  , -116.82195  ,  -35.10863  ,   65.794655 ,
         -66.88236  ],
       [-262.6041   , -184.733    ,  -73.09801  ,  133.00174  ,
         -51.110992 ],
       [-143.66011  ,  -96.94547  ,  -20.386663 ,   82.67798  ,
         -75.36298  ],
       [-221.4947   ,  171.18665  ,  -43.155212 , -116.659096 ,
         -61.94471  ]], dtype=float32)

In [14]:
test_pred_bool = (test_pred > 1)
test_predictions = test_pred_bool.astype(int)
test_predictions

array([[0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0]])

In [15]:
from PIL import Image

In [16]:
image = Image.open('test_mug.jpg')

image = np.expand_dims(image, axis=0)
image = tf.concat(image, axis=0)
image = tf.image.resize(image, [INPUT_SIZE, INPUT_SIZE])
image = tf.cast(image, tf.float32)
image /= 255
image

<tf.Tensor: shape=(1, 224, 224, 3), dtype=float32, numpy=
array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]]], dty

In [17]:
prediction = model.predict(image)
prediction

array([[-109.40728 ,  -48.67871 ,   -9.58495 ,   -8.810301,  -93.785065]],
      dtype=float32)

In [18]:
pred_bool = (prediction > 1)
pred_bool = pred_bool.astype(int)
pred_bool

array([[0, 0, 0, 0, 0]])